In [1]:
import numpy as np
import scipy as sp
import os 
import sys
import importlib

In [2]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu
import my_steven_free_utils as msfreeu
import my_classification_utils as mcu
import my_meal_detection_utils as mmdu
import my_data_process_utils as mdpu
#implortlib.reload(mmdu)

In [3]:
annots = mfileu.read_file('data', 'free_data_steven_annots.pkl')
#lab_data = mfileu.read_file('data', 'lab_data_steven.pkl')
free_data = mfileu.read_file('data', 'free_data_steven_right.pkl')

In [4]:
all_proba_bite = []
for subj in range(len(annots)):
    subj_proba_bite = []
    for sess in range(len(annots[subj])):
        r1 = mfileu.read_file('baseline_test_proba_gatech', 'bite_free_'+str(subj)+'_'+str(sess)+".pkl")
        subj_proba_bite.append(r1)
        
    all_proba_bite.append(subj_proba_bite)        

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

def get_result(subj, sess, minPts, eps, plot=False):
    result = []
    
    #print("Subj, sess: ", subj, sess)

    a = annots[subj][sess]        
    a = msfreeu.process_anntos(len(free_data[subj][sess]), a)
    #print("Meal, snack count:", np.sum(a[:,2]==1), np.sum(a[:,2]==2))
    
    proba = all_proba_bite[subj][sess]
    indices = proba[:, 0]+48
    proba = proba[:, -1]    
    bite_indices = indices[proba>=0.5]
    
    clusters = cluster_dbscan(bite_indices, minPts=minPts, eps=eps)    
    result = mmdu.get_meal_detection_results(a, clusters=clusters)
    
    ###############################################
    if not plot:
        return result

    idf = 16*60*60
    fig = plt.figure(figsize=(20,8))
    ax = plt.subplot(111)        
    ax.scatter(bite_indices/idf, np.zeros((len(bite_indices)),)+1, marker='x', s=10, color='blue')                
    
    clrs = ['', 'blue', 'green', 'red', 'black']
    for i in range(len(a)):
        si = a[i, 0]/idf
        ei = a[i, 1]/idf
        mt = a[i, 2]            
        ax.plot([si, ei], [0.5, 0.5], color=clrs[mt], linewidth=5)

    for i in range(len(clusters)):
        si = clusters[i, 0]/idf
        ei = clusters[i, 1]/idf            
        ax.plot([si, ei], [0.75, 0.75], color='black', linewidth=5)
    

    plt.title("Subj, sess:{}, {}".format(subj, sess))
    plt.xlabel('Time (Hour)', fontsize=20)        
    plt.ylim([0, 2])
    plt.grid(True)
    plt.show()        

    return result

In [7]:
def get_metric_results(minPts, eps):
    gts, acovs, clcovs = [], [], []
    for subj in range(len(annots)):
        for sess in range(len(annots[subj])):
                              
            gt, ac, clc = get_result(subj, sess, minPts=minPts, eps=eps, plot=False)
            assert len(gt) == len(ac)

            gt = mdpu.add_subj_sess_to_array(gt, subj, sess, at_begin=False)
            ac = mdpu.add_subj_sess_to_array(ac, subj, sess, at_begin=False)
            clc = mdpu.add_subj_sess_to_array(clc, subj, sess, at_begin=False)

            gts = gt if len(gts)==0 else np.concatenate((gts, gt))
            acovs = ac if len(acovs)==0 else np.concatenate((acovs, ac))
            clcovs = clc if len(clcovs)==0 else np.concatenate((clcovs, clc))
            
    res = mmdu.get_metric_results(gts, acovs, clcovs)
    return res

In [11]:
res ={}
for minPts in range(1, 11):
    for eps in range(10, 601, 10):
        print(minPts, eps, end=' | ')
        res[(minPts, eps)] = get_metric_results(minPts, eps*16)

mfileu.write_file('final_results', 'gatech.pkl', res)

1 10 | 1 20 | 1 30 | 1 40 | 1 50 | 1 60 | 1 70 | 1 80 | 1 90 | 1 100 | 1 110 | 1 120 | 1 130 | 1 140 | 1 150 | 1 160 | 1 170 | 1 180 | 1 190 | 1 200 | 1 210 | 1 220 | 1 230 | 1 240 | 1 250 | 1 260 | 1 270 | 1 280 | 1 290 | 1 300 | 1 310 | 1 320 | 1 330 | 1 340 | 1 350 | 1 360 | 1 370 | 1 380 | 1 390 | 1 400 | 1 410 | 1 420 | 1 430 | 1 440 | 1 450 | 1 460 | 1 470 | 1 480 | 1 490 | 1 500 | 1 510 | 1 520 | 1 530 | 1 540 | 1 550 | 1 560 | 1 570 | 1 580 | 1 590 | 1 600 | 2 10 | 2 20 | 2 30 | 2 40 | 2 50 | 2 60 | 2 70 | 2 80 | 2 90 | 2 100 | 2 110 | 2 120 | 2 130 | 2 140 | 2 150 | 2 160 | 2 170 | 2 180 | 2 190 | 2 200 | 2 210 | 2 220 | 2 230 | 2 240 | 2 250 | 2 260 | 2 270 | 2 280 | 2 290 | 2 300 | 2 310 | 2 320 | 2 330 | 2 340 | 2 350 | 2 360 | 2 370 | 2 380 | 2 390 | 2 400 | 2 410 | 2 420 | 2 430 | 2 440 | 2 450 | 2 460 | 2 470 | 2 480 | 2 490 | 2 500 | 2 510 | 2 520 | 2 530 | 2 540 | 2 550 | 2 560 | 2 570 | 2 580 | 2 590 | 2 600 | 3 10 | 3 20 | 3 30 | 3 40 | 3 50 | 3 60 | 3 70 | 3 80 | 3 

C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils\my_meal_detection_utils.py:225: RuntimeWarning: invalid value encountered in long_scalars
  res['precision'] = res['tp']/(res['tp'] + res['fp'])


6 300 | 6 310 | 6 320 | 6 330 | 6 340 | 6 350 | 6 360 | 6 370 | 6 380 | 6 390 | 6 400 | 6 410 | 6 420 | 6 430 | 6 440 | 6 450 | 6 460 | 6 470 | 6 480 | 6 490 | 6 500 | 6 510 | 6 520 | 6 530 | 6 540 | 6 550 | 6 560 | 6 570 | 6 580 | 6 590 | 6 600 | 7 10 | 7 20 | 7 30 | 7 40 | 7 50 | 7 60 | 7 70 | 7 80 | 7 90 | 7 100 | 7 110 | 7 120 | 7 130 | 7 140 | 7 150 | 7 160 | 7 170 | 7 180 | 7 190 | 7 200 | 7 210 | 7 220 | 7 230 | 7 240 | 7 250 | 7 260 | 7 270 | 7 280 | 7 290 | 7 300 | 7 310 | 7 320 | 7 330 | 7 340 | 7 350 | 7 360 | 7 370 | 7 380 | 7 390 | 7 400 | 7 410 | 7 420 | 7 430 | 7 440 | 7 450 | 7 460 | 7 470 | 7 480 | 7 490 | 7 500 | 7 510 | 7 520 | 7 530 | 7 540 | 7 550 | 7 560 | 7 570 | 7 580 | 7 590 | 7 600 | 8 10 | 8 20 | 8 30 | 8 40 | 8 50 | 8 60 | 8 70 | 8 80 | 8 90 | 8 100 | 8 110 | 8 120 | 8 130 | 8 140 | 8 150 | 8 160 | 8 170 | 8 180 | 8 190 | 8 200 | 8 210 | 8 220 | 8 230 | 8 240 | 8 250 | 8 260 | 8 270 | 8 280 | 8 290 | 8 300 | 8 310 | 8 320 | 8 330 | 8 340 | 8 350 | 8 360 | 8 